# Introduction to Pandas

_pandas_ is an open source Python library for data analysis.
Thanks to Gaia Saveri for this notebook.

## Data structures 

Pandas introduces two new data structures: **Series** and **Data frame**. 

### Series

A Series in a one-dimensional object (similar to an array, list, or column in a table). A labeled index is assigned to each item in the Series (the default are 0-N indeces, being N the length of the Series minus one).

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# create a series with a list
s = pd.Series([7, 'Hello world', 42.26])
print("Series with default index: ")
print(s)

In [ ]:
# create a series with a user-defined index
series_with_index = pd.Series([2, True, 'Hey'], index=['A', 'B', 'C'])
print("\nSeries with user-defined index: ")
print(series_with_index)

In [ ]:
# create a series from dictionary
# in this case the index is made by the keys of the dictionary
d = {'One':1, 'Two':2, 'Three':3}
series_from_dict = pd.Series(d)
print("\nSeries from dictionary:")
print(series_from_dict)

In [ ]:
# indexing series
dd = {'Chicago':1000, 'New York':1300, 'Portland':900, 'San Francisco':1100,
     'Austin':450, 'Boston':None}
cities = pd.Series(dd)
print("\nCities Series:")
print(cities)

In [ ]:
# use the index to select specific items
c = cities[['New York', 'San Francisco', 'Boston']]
c

In [ ]:
# use boolean indexing for selection
cc = cities[cities<1000]
cc

In [ ]:
# change the value in a Series
cities['Austin'] = 500 
cities

In [ ]:
# add two series together
# returns a union of the two series with the addition occurring on the shared index values
# NaN (Not a Number) on the others 
s1 = pd.Series([1, 2, 3], index=['A', 'B', 'C'])
s2 = pd.Series([4, 5, 6], index=['A', 'D', 'C'])

print("\ns1: ")
print(s1)
print("\ns2: ")
print(s2)
print("\ns1 + s2: ")
ss = s1 + s2
print(ss)

In [ ]:
# null checking
# using notnull: returns a boolean series indicating which values aren't null
not_null = cities.notnull()
print("\nNon-null elements in cities (notnull):")
print(not_null)

# using isnull: returns a boolean series indicating which values are null
is_null = cities.isnull()
is_null_cities = cities[is_null]
print("\nNull elements in cities (isnull): ")
print(is_null_cities)

### DataFrame

A DataFrame is a tabular data structure made of rows and columns. It can be thought of as a group of Series objects that share an index (the columns' name). 

#### Reading data

In order to create manually a DataFrame we can pass a dictionary of lists to the DataFrame constructor:

In [ ]:
data = {'letters':['A', 'B', 'C', 'D'],
       'numbers':[1, 2, 3, 4],
       'spelling': ['One', 'Two', 'Three', 'Four']}
letters_numbers = pd.DataFrame(data, columns=['letters', 'numbers', 'spelling'])
letters_numbers

More often we have a dataset that we want to read into a DataFrame.

The most common formats are *CSV* and *excel*, they are read using the functions `read_csv` and `read_excel` respectively.

Donwload the data from https://archive-beta.ics.uci.edu/dataset/602/dry+bean+dataset (Dry Bean Dataset, 2020, UCI Machine Learning Repository) and move it to the current directory.

In [ ]:
# import excel dataset
beans = pd.read_excel('../Datasets/Dry_Bean_Dataset.xlsx')
# head(n) shows the first n rows (default is n=5)
beans.head()

In [ ]:
# import csv dataset
path_csv = 'https://raw.githubusercontent.com/GaiaSaveri/intro-to-ml/main/data/ign.csv'
reviews = pd.read_csv(path_csv)
reviews.head(3)

#### Indexing

Pandas supports several methods for indexing data in a table:

* `iloc` (integer based indexing): refers to rows and columns by their position, expressed as an integer starting from zero;
* `loc` (label based indexing): uses row and column name;
* Boolean indexing: uses boolean masks. Row names are stored in the field `index` of a data frame, columns names are stored in the field `columns`. 

In [ ]:
col = reviews.columns # columns' names
reviews = reviews.iloc[:,1:] # ignoring the column 0
reviews.head() # problem fixed!

In [ ]:
# select a subset of rows and columns of the dataframe
reviews.iloc[0:10, 0:2] # first ten rows, first two columns

In [ ]:
# select a subset of rows of the dataframe
restricted_reviews = reviews.loc[10:20] # rows from ten to twenty
restricted_reviews

In [ ]:
# select a subset of rows and columns using columns' names
reviews.loc[0:10, ["title", "score"]]

In [ ]:
# select a subset of columns by their names keeping all rows
reviews[["title", "score"]]

In [ ]:
# indexing using boolean mask
mask = reviews["score"] >= 9.5 
mask # series of booleans

In [ ]:
# retrieve title of the games having a score >=9.5
cool_games = reviews[mask]["title"]
print("Fraction of cool games: ")
# len returns the number of rows
print(len(cool_games)/len(reviews["title"]))
cool_games

The following are some functions that can be used to get information about a DataFrame and do some maths with numeric columns:

In [ ]:
# info about each column
reviews.info() 

In [ ]:
# statistical infos about the column score
reviews["score"].describe() 

In [ ]:
# mean of each numeric column
reviews.mean() 

In [ ]:
# median of each numeric column
reviews.median() 

In [ ]:
# arithmetic manipulation of the score column
reviews["score"]/2*3

#### Summarising data

In [ ]:
# unique: returns unique values of series object
reviews["platform"].unique()

In [ ]:
# value_counts: returns a series containing counts of unique values
reviews["release_year"].value_counts(dropna=False)

#### Grouping data

Grouping data allows to aggregate data with respect to the values of one column, typically a category, even hierarchically.

It is useful especially in data analysis problems, in which one often breaks the problem apart into manageble pieces, and then put everything back together again.

The method `groupby` returns a collection of dataframes, which can be iterated on. 

In [ ]:
grouped = reviews.groupby('genre')
grouped.size()

In [ ]:
# iterability/dataframe iteration
for name, group in grouped:
    print(f"{name} - {group.shape} \n {type(group)}")

In [ ]:
# another example of grouping
reviews.groupby('score_phrase')['score'].mean()

#### Applying functions to Series and DataFrames

With the method `apply` of the Series object we can apply a (python/numpy/user-defined/lambda) function to each element of the Series. 

In [ ]:
# transform (encode) the column gender to a numeric one
def editor_code(gstr):
    if(pd.isnull(gstr)):
        return gstr
    elif gstr == "Y":
        return 1
    else: # gstr == "N"
        return 0

# apply function
reviews['editors_choice'] = reviews['editors_choice'].apply(editor_code)
# check the result
reviews['editors_choice'].value_counts(dropna=False)

#### Aggregating data

Numerical columns in grouped structures can be aggregated, in order for example to visualize their mean, sum, etc.. by calling the `agg` function.

In [ ]:
grouped.agg(np.mean)

In [ ]:
# we can also pass a list of aggregating functions
gr_genre = grouped["score"].agg([np.mean, np.std])
gr_genre

#### Plotting data 

The `plot` method of `DataFrame` is the way in which data are plotted in Pandas.

The option `kind` allows the selection of the plot type (e.g. hist, bar).

In [ ]:
# histogram
reviews["score"].plot(kind='hist');
# equivalent:
# reviews["score"].hist() 

In [ ]:
# bar plot
gr_genre.plot(kind="bar");